In [32]:
pip install pulp

Note: you may need to restart the kernel to use updated packages.


In [33]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
from pulp import * 
import seaborn as sns

In [34]:
data = pd.read_csv('nutrition.csv').drop('Unnamed: 0',axis=1)
data.head()

,name,serving_size,calories,total_fat,protein,carbohydrate,fiber,fat
0,Cornstarch,100 g,381,0.1g,0.26 g,91.27 g,0.9 g,0.05 g
1,"Nuts, pecans",100 g,691,72g,9.17 g,13.86 g,9.6 g,71.97 g
2,"Eggplant, raw",100 g,25,0.2g,0.98 g,5.88 g,3.0 g,0.18 g
3,"Teff, uncooked",100 g,367,2.4g,13.30 g,73.13 g,8.0 g,2.38 g
4,"Sherbet, orange",100 g,144,2g,1.10 g,30.40 g,1.3 g,2.00 g


In [35]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8789 entries, 0 to 8788
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   name          8789 non-null   object
 1   serving_size  8789 non-null   object
 2   calories      8789 non-null   int64 
 3   total_fat     8789 non-null   object
 4   protein       8789 non-null   object
 5   carbohydrate  8789 non-null   object
 6   fiber         8789 non-null   object
 7   fat           8789 non-null   object
dtypes: int64(1), object(7)
memory usage: 549.4+ KB


In [36]:
data['carbohydrate'] = np.array([data['carbohydrate'].tolist()[i].split(' ') for i in range(len(data))])[:,0].astype('float')
data['protein'] = np.array([data['protein'].tolist()[i].split(' ') for i in range(len(data))])[:,0].astype('float')
data['total_fat'] = np.array([data['total_fat'].tolist()[i].split('g') for i in range(len(data))])[:,0].astype('float')

In [37]:
data['total_fat']

0        0.1
1       72.0
2        0.2
3        2.4
4        2.0
        ... 
8784     3.5
8785     8.9
8786    23.0
8787     3.0
8788     3.0
Name: total_fat, Length: 8789, dtype: float64

In [38]:
data['protein']

0        0.26
1        9.17
2        0.98
3       13.30
4        1.10
        ...  
8784    23.45
8785    29.59
8786    16.74
8787    23.37
8788    23.37
Name: protein, Length: 8789, dtype: float64

In [39]:
def build_nutritional_values(kg,calories):
    protein_calories = kg*4
    res_calories = calories-protein_calories
    carb_calories = calories/2.
    fat_calories = calories-carb_calories-protein_calories
    res = {'Protein Calories':protein_calories,'Carbohydrates Calories':carb_calories,'Fat Calories':fat_calories}
    return res

In [40]:
build_nutritional_values(70,2000)

{'Protein Calories': 280,
 'Carbohydrates Calories': 1000.0,
 'Fat Calories': 720.0}

In [41]:
def extract_gram(table):
    protein_grams = table['Protein Calories']/4.
    carbs_grams = table['Carbohydrates Calories']/4.
    fat_grams = table['Fat Calories']/9.
    res = {'Protein Grams':protein_grams, 'Carbohydrates Grams':carbs_grams,'Fat Grams':fat_grams}
    return res

In [42]:
print(build_nutritional_values(70,1500))
print(extract_gram(build_nutritional_values(70,1500)))

{'Protein Calories': 280, 'Carbohydrates Calories': 750.0, 'Fat Calories': 470.0}
{'Protein Grams': 70.0, 'Carbohydrates Grams': 187.5, 'Fat Grams': 52.22222222222222}


In [43]:
week_days = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
split_values_day = np.linspace(0,len(data),8).astype(int)
split_values_day[-1] = split_values_day[-1]-1
def random_dataset_day():
    frac_data = data.sample(frac=1).reset_index().drop('index',axis=1)
    day_data = []
    for s in range(len(split_values_day)-1):
        day_data.append(frac_data.loc[split_values_day[s]:split_values_day[s+1]])
    return dict(zip(week_days,day_data))

In [44]:
meals = ['Snack 1','Snack 2','Breakfast','Lunch','Dinner']
split_values_meal = np.linspace(0,split_values_day[1],len(meals)+1).astype(int)
split_values_meal[-1] = split_values_meal[-1]-1
def random_dataset_meal(data_day):
    frac_data = data_day.sample(frac=1).reset_index().drop('index',axis=1)
    meal_data = []
    for s in range(len(split_values_meal)-1):
        meal_data.append(frac_data.loc[split_values_meal[s]:split_values_meal[s+1]])
    return dict(zip(meals,meal_data))

In [60]:
random_dataset_day()['Monday']

,name,serving_size,calories,total_fat,protein,carbohydrate,fiber,fat
0,"Pasta mix, unprepared, Italian lasagna",100 g,356,2.0,10.90,73.77,2.8 g,1.97 g
1,"Oil, uses similar to high quality cocoa butter...",100 g,884,100.0,0.00,0.00,0.0 g,100.00 g
2,"Beans, without salt, drained, boiled, cooked, ...",100 g,33,0.6,4.83,4.72,0,0.58 g
3,"Beef, pan-broiled, cooked, patty, 95% lean mea...",100 g,164,5.9,25.80,0.00,0.0 g,5.94 g
4,"Beef, roasted, cooked, select, trimmed to 1/8""...",100 g,202,9.7,28.72,0.00,0.0 g,9.68 g
...,...,...,...,...,...,...,...,...
1251,"Beef, pan-browned, cooked, crumbles, 85% lean ...",100 g,256,15.0,27.73,0.00,0.0 g,15.30 g
1252,"GLUTINO, Chocolate Vanilla Creme, Gluten Free ...",100 g,474,18.0,2.19,76.03,2.5 g,17.89 g
1253,"Chicken, stewed, cooked, meat and skin, wing, ...",100 g,249,17.0,22.78,0.00,0.0 g,16.82 g
1254,"KEEBLER, Sweet Cremes Cookies",100 g,469,19.0,4.50,71.00,1.3 g,19.30 g


In [63]:
random_dataset_day()['Tuesday']

,name,serving_size,calories,total_fat,protein,carbohydrate,fiber,fat
1255,"Sauce, TABASCO, pepper, ready-to-serve",100 g,12,0.8,1.29,0.80,0.6 g,0.76 g
1256,"Game meat, roasted, cooked, antelope",100 g,150,2.7,29.45,0.00,0.0 g,2.67 g
1257,"TACO BELL, Nachos",100 g,350,22.0,4.32,34.91,3.2 g,21.50 g
1258,"Cranberry, raw (Alaska Native), low bush or li...",100 g,55,0.5,0.40,12.20,0,0.50 g
1259,"Beef, roasted, cooked, choice, trimmed to 1/8""...",100 g,240,15.0,27.18,0.00,0.0 g,14.55 g
...,...,...,...,...,...,...,...,...
2507,"KELLOGG'S, Blueberry, Pancakes, EGGO",100 g,245,7.4,5.30,39.90,1.0 g,7.40 g
2508,"Fish, liver (Alaska Native), chinook, king, sa...",100 g,156,8.0,16.60,4.30,0,8.00 g
2509,"Snacks, mixed flavors, chewy, KASHI GOLEAN, gr...",100 g,390,7.7,16.67,63.42,7.7 g,7.69 g
2510,"Beef, fast roasted, cooked, separable lean onl...",100 g,239,13.0,30.09,0.00,0.0 g,13.22 g


In [65]:
random_dataset_day()['Wednesday']

,name,serving_size,calories,total_fat,protein,carbohydrate,fiber,fat
2511,"Restaurant, chicken chow mein, Chinese",100 g,85,2.8,6.76,8.29,1.0 g,2.80 g
2512,"Peanut spread, reduced sugar",100 g,650,55.0,24.80,14.23,7.8 g,54.89 g
2513,"Spices, fennel seed",100 g,345,15.0,15.80,52.29,39.8 g,14.87 g
2514,"Fish, dry heat, cooked, sheepshead",100 g,126,1.6,26.02,0.00,0.0 g,1.63 g
2515,"Beef, grilled, cooked, choice, trimmed to 0"" f...",100 g,184,8.1,26.07,0.00,0.0 g,8.11 g
...,...,...,...,...,...,...,...,...
3762,"Veal, cooked, separable fat, composite of trim...",100 g,642,67.0,9.42,0.00,0.0 g,66.74 g
3763,"BURGER KING, no cheese, DOUBLE WHOPPER",100 g,252,16.0,13.94,13.74,1.4 g,15.66 g
3764,"Pancakes, prepared from recipe, blueberry",100 g,222,9.2,6.10,29.00,0,9.20 g
3765,"Soup, vegetable broth, SWANSON",100 g,6,0.1,0.23,1.02,0.0 g,0.07 g


In [ ]:
days_data = random_dataset_day()
def model(day,kg,calories):
    G = extract_gram(build_nutritional_values(kg,calories))
    E = G['Carbohydrates Grams']
    F = G['Fat Grams']
    P = G['Protein Grams']
    day_data = days_data[day]
    day_data = day_data[day_data.calories!=0]
    food = day_data.name.tolist()
    c  = day_data.calories.tolist()
    x  = pulp.LpVariable.dicts( "x", indices = food, lowBound=0, upBound=1.5, cat='Continuous', indexStart=[] )
    e = day_data.carbohydrate.tolist()
    f = day_data.total_fat.tolist()
    p = day_data.protein.tolist()
    prob  = pulp.LpProblem( "Diet", LpMinimize )
    prob += pulp.lpSum( [x[food[i]]*c[i] for i in range(len(food))]  )
    prob += pulp.lpSum( [x[food[i]]*e[i] for i in range(len(x)) ] )>=E
    prob += pulp.lpSum( [x[food[i]]*f[i] for i in range(len(x)) ] )>=F
    prob += pulp.lpSum( [x[food[i]]*p[i] for i in range(len(x)) ] )>=P
    prob.solve()
    variables = []
    values = []
    for v in prob.variables():
        variable = v.name
        value = v.varValue
        variables.append(variable)
        values.append(value)
    values = np.array(values).round(2).astype(float)
    sol = pd.DataFrame(np.array([food,values]).T, columns = ['Food','Quantity'])
    sol['Quantity'] = sol.Quantity.astype(float)
    return sol

In [ ]:
sol_monday = model('Monday',70,1500)
print(sol_monday)

In [ ]:
sol_monday = sol_monday[sol_monday['Quantity']!=0.0]
sol_monday.Quantity = sol_monday.Quantity*100
sol_monday = sol_monday.rename(columns={'Quantity':'Quantity (g)'})
sol_monday

In [ ]:
def model(prob, kg,calories,meal,data):
    G = extract_gram(build_nutritional_values(kg,calories))
    E = G['Carbohydrates Grams']
    F = G['Fat Grams']
    P = G['Protein Grams']
    day_data = data
    day_data = day_data[day_data.calories!=0]
    food = day_data.name.tolist()
    c  = day_data.calories.tolist()
    x  = pulp.LpVariable.dicts( "x", indices = food, lowBound=0, upBound=1.5, cat='Continuous', indexStart=[] )
    e = day_data.carbohydrate.tolist()
    f = day_data.total_fat.tolist()
    p = day_data.protein.tolist()
#    prob  = pulp.LpProblem( "Diet", LpMinimize )
    div_meal = meal_split[meal]
    prob += pulp.lpSum( [x[food[i]]*c[i] for i in range(len(food))]  )
    prob += pulp.lpSum( [x[food[i]]*e[i] for i in range(len(x)) ] )>=E*div_meal
    prob += pulp.lpSum( [x[food[i]]*f[i] for i in range(len(x)) ] )>=F*div_meal
    prob += pulp.lpSum( [x[food[i]]*p[i] for i in range(len(x)) ] )>=P*div_meal
    prob.solve()
    variables = []
    values = []
    for v in prob.variables():
        variable = v.name
        value = v.varValue
        variables.append(variable)
        values.append(value)
    values = np.array(values).round(2).astype(float)
    sol = pd.DataFrame(np.array([food,values]).T, columns = ['Food','Quantity'])
    sol['Quantity'] = sol.Quantity.astype(float)
    sol = sol[sol['Quantity']!=0.0]
    sol.Quantity = sol.Quantity*100
    sol = sol.rename(columns={'Quantity':'Quantity (g)'})
    return sol
def total_model(kg,calories):
    result = []
    for day in week_days:
        prob  = pulp.LpProblem( "Diet", LpMinimize )
        print('Building a model for day %s \n'%(day))
        result.append(model(prob,day,kg,meal,data))
    return dict(zip(week_days,result))

In [ ]:
meal_split = {'Snack 1': 0.10, 'Snack 2':0.10,'Breakfast': 0.15,'Lunch':0.35, 'Dinner':0.30}
labels = []
sizes = []

for x, y in meal_split.items():
    labels.append(x)
    sizes.append(y)

# Plot
plt.figure(figsize=(10,10))
plt.pie(sizes, labels=labels,explode = [0.,0.,0.08,0.1,0.1],textprops={'fontsize': 14})

plt.axis('equal')
plt.show()

In [48]:
split_size = np.linspace(0,len(data),8)


In [73]:

def better_model(kg, calories):
    days_data = random_dataset_day()
    res_model = []
    for day in week_days:
        day_data = days_data[day]
        meals_data = random_dataset_meal(day_data)
        meal_model = []
        for meal in meals:
            meal_data = meals_data[meal]
            prob = pulp.LpProblem("Diet", LpMinimize)
            sol_model = model(prob, kg, calories, meal, meal_data)
            meal_model.append(sol_model)
        res_model.append(meal_model)
    
    unpacked = []
    for i in range(len(res_model)):
        unpacked.append(dict(zip(meals, res_model[i])))
    
    unpacked_tot = dict(zip(week_days, unpacked))
    return unpacked_tot




In [74]:
diet = better_model(70, 1500)

In [75]:
diet['Monday']

{'Snack 1':                                                   Food  Quantity (g)
 51    Crackers, saltines (includes oyster, soda, soup)         150.0
 164  Lamb, roasted, cooked, trimmed to 1/8" fat, se...           5.0
 178                                      Apricots, raw           4.0
 229             Fish, dry heat, cooked, northern, pike          27.0,
 'Snack 2':                                                   Food  Quantity (g)
 10   Pork, unheated, separable lean and fat, bonele...           5.0
 14   Juice, with added ascorbic acid and calcium, g...         104.0
 114  Pork, separable lean only, unheated, bone-in, ...         101.0,
 'Breakfast':                                                   Food  Quantity (g)
 125                   Emu, broiled, cooked, fan fillet          67.0
 172  Cereals ready-to-eat, DORA THE EXPLORER, GENER...         132.0
 187      Candies, caramel with nuts, chocolate covered          10.0,
 'Lunch':                                           

In [76]:
diet['Saturday']['Lunch']

,Food,Quantity (g)
4,"Sweet potato, without skin, boiled, cooked",150.0
68,"Chokecherries, pitted (Northern Plains Indians...",6.0
155,"Cereals, dry, plain, fortified, instant, oats",150.0
230,"Lamb, raw, ground lamb, imported, New Zealand",99.0
235,"Cereals, dry, WHEATENA",52.0


In [77]:
diet['Sunday']['Breakfast']

,Food,Quantity (g)
108,"Veal, braised, cooked, thymus, variety meats a...",150.0
160,"KEEBLER, Mini Fudge Grahams Cookies, FUDGE SHO...",63.0
164,"Beef, raw, choice, trimmed to 0"" fat, separabl...",5.0
220,"Snacks, chocolate chip, hard, granola bars",43.0


In [78]:
diet['Monday']['Snack 1']

,Food,Quantity (g)
51,"Crackers, saltines (includes oyster, soda, soup)",150.0
164,"Lamb, roasted, cooked, trimmed to 1/8"" fat, se...",5.0
178,"Apricots, raw",4.0
229,"Fish, dry heat, cooked, northern, pike",27.0
